In [1]:
import os
import numpy as np
import pandas as pd
import librosa
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping , ModelCheckpoint
from tensorflow.keras.models import load_model
from datetime import datetime

In [2]:
data_dir = '/Users/irk2w/Desktop/T5/sounds'
classes = ['Civil', 'Police', 'Trafic', 'ambulance']

In [3]:
def extract_features(file_path, target_shape=(128, 128)):
    audio_data, sample_rate = librosa.load(file_path, sr=None)

    # Data Augmentation
    pitch_shifted = librosa.effects.pitch_shift(audio_data, sr=sample_rate, n_steps=4)
    time_stretched = librosa.effects.time_stretch(audio_data, rate=1.5)

    # Original Features
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    # Augmented Features
    mel_spectrogram_pitch = librosa.feature.melspectrogram(y=pitch_shifted, sr=sample_rate)
    mel_spectrogram_stretch = librosa.feature.melspectrogram(y=time_stretched, sr=sample_rate)

    # Resizing
    mel_spectrogram_resized = resize(mel_spectrogram, target_shape)
    mel_spectrogram_pitch_resized = resize(mel_spectrogram_pitch, target_shape)
    mel_spectrogram_stretch_resized = resize(mel_spectrogram_stretch, target_shape)

    return mel_spectrogram_resized, mel_spectrogram_pitch_resized, mel_spectrogram_stretch_resized


In [4]:
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []

    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                original, pitch, stretch = extract_features(file_path, target_shape)
                # Append original features
                data.append(original)
                labels.append(i)
                # Append augmented features
                data.append(pitch)
                labels.append(i)
                data.append(stretch)
                labels.append(i)

    return np.array(data), np.array(labels)

In [5]:
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

/var/folders/vt/q25n1vbx5bsghn3n8y3wzftc0000gn/T/ipykernel_19335/3740087319.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sample_rate = librosa.load(file_path, sr=None)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [49]:
data, l = load_and_preprocess_data(data_dir, classes)

/var/folders/vt/q25n1vbx5bsghn3n8y3wzftc0000gn/T/ipykernel_3270/3740087319.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sample_rate = librosa.load(file_path, sr=None)


In [6]:
pd.Series(l).value_counts()

NameError: name 'l' is not defined

In [18]:
model = Sequential()
model.add(Dense(256, input_shape=(128, 128), activation='relu'))  # Adjust the input_shape according to your data
model.add(Dropout(0.5))
model.add(Dense(192, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(LSTM(64))
model.add(Dense(len(classes), activation='softmax'))

In [19]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [20]:
# Display model architecture summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128, 256)          33024     
                                                                 
 dropout_4 (Dropout)         (None, 128, 256)          0         
                                                                 
 dense_9 (Dense)             (None, 128, 192)          49344     
                                                                 
 dropout_5 (Dropout)         (None, 128, 192)          0         
                                                                 
 dense_10 (Dense)            (None, 128, 128)          24704     
                                                                 
 dropout_6 (Dropout)         (None, 128, 128)          0         
                                                                 
 dense_11 (Dense)            (None, 128, 64)          

In [21]:
# Calculate pre-training accuracy
score = model.evaluate(X_test, y_test, verbose=0)
accuracy = 100*score[1]

In [22]:
# Print pre-training accuracy
print("Pre-training accuracy: %.4f%%" % accuracy)

Pre-training accuracy: 20.1923%


In [12]:
# Train the model
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.h5',
                               verbose=1, save_best_only=True)

In [13]:
start = datetime.now()

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=1500, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/1500
3/4 [=====================>........] - ETA: 0s - loss: 1.3866 - accuracy: 0.2344
Epoch 1: val_loss did not improve from 0.58109
4/4 [==============================] - 1s 91ms/step - loss: 1.3867 - accuracy: 0.2306 - val_loss: 1.3879 - val_accuracy: 0.2019
Epoch 2/1500
3/4 [=====================>........] - ETA: 0s - loss: 1.3861 - accuracy: 0.2474
Epoch 2: val_loss did not improve from 0.58109
4/4 [==============================] - 0s 86ms/step - loss: 1.3863 - accuracy: 0.2379 - val_loss: 1.3876 - val_accuracy: 0.2308
Epoch 3/1500
3/4 [=====================>........] - ETA: 0s - loss: 1.3860 - accuracy: 0.2682
Epoch 3: val_loss did not improve from 0.58109
4/4 [==============================] - 0s 84ms/step - loss: 1.3861 - accuracy: 0.2646 - val_loss: 1.3875 - val_accuracy: 0.2019
Epoch 4/1500
3/4 [=====================>........] - ETA: 0s - loss: 1.3859 - accuracy: 0.2682
Epoch 4: val_loss did not improve from 0.58109
4/4 [==============================] - 0s 87ms/step 

In [15]:
# Evaluating the model on the training and testing set
score = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

Training Accuracy:  0.9902912378311157


In [16]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Testing Accuracy:  0.817307710647583


In [64]:
model.save('911_class_best.h5')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
from tensorflow.keras.models import load_model

def predict_audio_class(file_path, model, classes):
    # Assuming the extract_features function is defined as above
    features = extract_features(file_path)[0]  # Use the original features for prediction
    features = np.expand_dims(features, axis=0)  # Reshaping to match model input

    # Make the prediction
    predictions = model.predict(features)
    predicted_class = np.argmax(predictions, axis=1)
    return classes[predicted_class[0]]

# Usage
#model = load_model("/Users/irk2w/Desktop/T5/911_model_don't_touch/911_class_test.h5")  # Load your model
audio_file_path = '/Users/irk2w/Downloads/civil1.wav'
predicted_class_name = predict_audio_class(audio_file_path, model, classes)
print(f"The model predicts that the audio file is a {predicted_class_name} sound.")

1/1 [==============================] - 0s 282ms/step
The model predicts that the audio file is a Trafic sound.


In [62]:
confusion_matrix(y_train.argmax(axis = 1), model.predict(X_train).argmax(axis = 1))

13/13 [==============================] - 0s 14ms/step


array([[100,   0,   0,   0],
       [  0,  99,   0,   0],
       [  0,   0, 108,   0],
       [  0,   0,   0, 105]])

In [63]:
confusion_matrix(y_test.argmax(axis = 1), model.predict(X_test).argmax(axis = 1))

4/4 [==============================] - 0s 10ms/step


array([[24,  0,  3,  2],
       [ 0, 30,  0,  0],
       [ 0,  1, 20,  0],
       [ 1,  5,  0, 18]])

In [69]:
len(y_train.argmax(axis = 1))

412

In [70]:
len(y_train)

412